In [13]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install requests beautifulsoup4 pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 498.8 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 217.4 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import re
from collections import defaultdict
import pandas as pd

# Constants
LEETCODE_DISCUSS_URL = "https://leetcode.com/discuss/general-discussion/"
SEARCH_QUERY = "google interview"
NUM_PAGES_TO_SCRAPE = 5  # Adjust based on how many pages you want to scrape

# Regex to match LeetCode question titles
QUESTION_REGEX = re.compile(r'https://leetcode\.com/problems/([^/]+)/')

# Store question frequencies
question_freq = defaultdict(int)

def scrape_discussion_page(driver, page_num):
    url = f"{LEETCODE_DISCUSS_URL}?currentPage={page_num}&orderBy=hot&query={SEARCH_QUERY}"
    driver.get(url)
    time.sleep(3)  # Wait for the page to load

    # Scroll to load all posts (adjust the range for more scrolling)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    posts = driver.find_elements(By.CLASS_NAME, 'discuss-post')
    questions = []
    for post in posts:
        content = post.text.lower()
        if "google" in content and "interview" in content:
            # Extract question titles from links
            links = post.find_elements(By.TAG_NAME, 'a')
            for link in links:
                href = link.get_attribute('href')
                if href:
                    match = QUESTION_REGEX.search(href)
                    if match:
                        question_title = match.group(1).replace('-', ' ')
                        questions.append(question_title)
    
    return questions

def scrape_leetcode_discussions():
    # Set up Selenium WebDriver
    service = Service(executable_path='/path/to/chromedriver')  # Replace with your ChromeDriver path
    driver = webdriver.Chrome(service=service)

    for page_num in range(1, NUM_PAGES_TO_SCRAPE + 1):
        print(f"Scraping page {page_num}...")
        questions = scrape_discussion_page(driver, page_num)
        for question in questions:
            question_freq[question] += 1
        time.sleep(2)  # Add delay to avoid rate-limiting

    driver.quit()

def get_top_questions(n=20):
    sorted_questions = sorted(question_freq.items(), key=lambda x: x[1], reverse=True)
    return sorted_questions[:n]

if __name__ == "__main__":
    scrape_leetcode_discussions()
    top_questions = get_top_questions()
    
    # Display results
    df = pd.DataFrame(top_questions, columns=["Question", "Frequency"])
    print(df)

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [6]:
pip install cloudscraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 11.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 437.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 287.5 kB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import cloudscraper
from bs4 import BeautifulSoup
from collections import Counter
import json
import time
import random

BASE_URL = "https://leetcode.com"
DISCUSSION_URL = "https://leetcode.com/discuss/interview-question?currentPage={}&orderBy=hot"

scraper = cloudscraper.create_scraper()

def get_google_questions(pages=5):
    question_counter = Counter()
    question_links = {}
    
    for page in range(1, pages + 1):
        url = DISCUSSION_URL.format(page)
        response = scraper.get(url)
        print(response.text)
        if response.status_code != 200:
            print(f"Failed to fetch page {page} with status code {response.status_code}")
            continue
        
        soup = BeautifulSoup(response.text, "html.parser")
        discussions = soup.select("div.truncate")  # Updated selector based on actual LeetCode structure
        
        for discussion in discussions:
            title_tag = discussion.select_one("a")
            if title_tag and title_tag.has_attr("href"):
                title = title_tag.text.strip()
                link = BASE_URL + title_tag["href"]
                
                if "Google" in title:
                    question_counter[title] += 1
                    question_links[title] = link
        
        time.sleep(random.uniform(2, 5))  # Adding delay to avoid detection
    
    return question_counter, question_links


def save_results(question_counter, question_links, filename="google_questions.json"):
    data = [{"title": title, "count": count, "link": question_links[title]} for title, count in question_counter.most_common()]
    
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)
    
    print(f"Results saved to {filename}")

if __name__ == "__main__":
    question_counter, question_links = get_google_questions(pages=3)
    save_results(question_counter, question_links)


<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background-s

In [19]:
!pip install selenium
!pip install webdriver-manager
!pip install chromedriver-autoinstaller


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from collections import Counter
import json
import time
import random
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()


BASE_URL = "https://leetcode.com"
DISCUSSION_URL = "https://leetcode.com/discuss/interview-question?currentPage={}&orderBy=hot"

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def get_google_questions(pages=5):
    question_counter = Counter()
    question_links = {}
    
    for page in range(1, pages + 1):
        url = DISCUSSION_URL.format(page)
        driver.get(url)
        time.sleep(random.uniform(3, 6))  # Adding delay to avoid detection
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        discussions = soup.select("div.truncate")  # Updated selector based on actual LeetCode structure
        
        for discussion in discussions:
            title_tag = discussion.select_one("a")
            if title_tag and title_tag.has_attr("href"):
                title = title_tag.text.strip()
                link = BASE_URL + title_tag["href"]
                
                if "Google" in title:
                    question_counter[title] += 1
                    question_links[title] = link
    
    return question_counter, question_links


def save_results(question_counter, question_links, filename="google_questions.json"):
    data = [{"title": title, "count": count, "link": question_links[title]} for title, count in question_counter.most_common()]
    
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)
    
    print(f"Results saved to {filename}")

if __name__ == "__main__":
    question_counter, question_links = get_google_questions(pages=3)
    save_results(question_counter, question_links)
    driver.quit()


ValueError: No chrome executable found on PATH